# Graph Classification

## Graph Classes

we use the following graph classes:
- Chemical Graphs (Molecules)
- Random
- Small World
- Scale Free

## Setup

We use pyg (pytorch-geometric) to generate the model to train.
The model is a GCN with 2 layers and 32 hidden units.

In [1]:
import torch_geometric
import torch

print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.mps.is_available())
print(torch_geometric.__version__)

1.13.1
None
True
2.2.0


In [2]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_networkx

# we need to read the dataset from the pickle file
import pickle

with open('dataset.pickle', 'rb') as f:
    dataset = pickle.load(f)

for key in dataset.keys():
    print(f"datasetkey: {key}, shape: {len(dataset[key])}")

# we need to convert the networkx graphs to pytorch geometric graphs

class_key = dict(zip(dataset.keys(), range(len(dataset.keys()))))
print(class_key)
pyg_dataset = []
for key in dataset.keys():
    for i in range(len(dataset[key])):
        graph = dataset[key][i]
        graph_tensor = from_networkx(graph, group_node_attrs=["label", "betweenness", "degree"])
        graph_tensor.y = torch.tensor([class_key[key]])
        pyg_dataset.append(graph_tensor)

datasetkey: random, shape: 225
datasetkey: smallworld, shape: 225
datasetkey: scalefree, shape: 98
datasetkey: complete, shape: 98
datasetkey: line, shape: 98
datasetkey: tree, shape: 24
datasetkey: randomtree, shape: 98
datasetkey: star, shape: 98
{'random': 0, 'smallworld': 1, 'scalefree': 2, 'complete': 3, 'line': 4, 'tree': 5, 'randomtree': 6, 'star': 7}


In [3]:
import torch
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx
import pickle


class MyOwnDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return ['data.pt']


    def process(self):
        # Read data into huge `Data` list.
        with open('dataset.pickle', 'rb') as f:
            dataset = pickle.load(f)

        class_key = dict(zip(dataset.keys(), range(len(dataset.keys()))))
        print(class_key)
        data_list = []
        for key in dataset.keys():
            for i in range(len(dataset[key])):
                graph = dataset[key][i]
                graph_tensor = from_networkx(graph, group_node_attrs=["label", "betweenness", "degree", "density"])
                graph_tensor.y = torch.tensor([class_key[key]])
                data_list.append(graph_tensor)

        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [4]:
import torch
from torch_geometric.datasets import TUDataset

dataset = MyOwnDataset(root='data/CustomData')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print(data.x)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...


{'random': 0, 'smallworld': 1, 'scalefree': 2, 'complete': 3, 'line': 4, 'tree': 5, 'randomtree': 6, 'star': 7}

Dataset: MyOwnDataset(964):
Number of graphs: 964
Number of features: 4
Number of classes: 8

Data(edge_index=[2, 12], x=[5, 4], y=[1])
tensor([[0.0000, 0.0833, 0.5000, 0.6000],
        [1.0000, 0.2500, 0.7500, 0.6000],
        [2.0000, 0.0833, 0.5000, 0.6000],
        [3.0000, 0.0000, 0.5000, 0.6000],
        [4.0000, 0.2500, 0.7500, 0.6000]])
Number of nodes: 5
Number of edges: 12
Average node degree: 2.40
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


In [5]:
torch.manual_seed(12345)
dataset = dataset.shuffle()

train_dataset = dataset[:600]
test_dataset = dataset[600:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 600
Number of test graphs: 364


In [6]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 45534], x=[2341, 4], y=[64], batch=[2341], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 42706], x=[2408, 4], y=[64], batch=[2408], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 43990], x=[2286, 4], y=[64], batch=[2286], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 35426], x=[2229, 4], y=[64], batch=[2229], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 53442], x=[2590, 4], y=[64], batch=[2590], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 36878], x=[2538, 4], y=[64], batch=[2538], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 40302], x=[2352, 4], y=[64], batch=[2352], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 52334], x=[2937, 4], y

In [7]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
from model.gcn import GCN

model = GCN(hidden_channels=64, num_classes=dataset.num_classes, num_node_features=dataset.num_node_features)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


confusion_matrix = torch.zeros(3, 3)
for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.2717, Test Acc: 0.2802
Epoch: 002, Train Acc: 0.2817, Test Acc: 0.3159
Epoch: 003, Train Acc: 0.2983, Test Acc: 0.3077
Epoch: 004, Train Acc: 0.3183, Test Acc: 0.3077
Epoch: 005, Train Acc: 0.3333, Test Acc: 0.3516
Epoch: 006, Train Acc: 0.5650, Test Acc: 0.5467
Epoch: 007, Train Acc: 0.6100, Test Acc: 0.5769
Epoch: 008, Train Acc: 0.7067, Test Acc: 0.6703
Epoch: 009, Train Acc: 0.7950, Test Acc: 0.7418
Epoch: 010, Train Acc: 0.7467, Test Acc: 0.7280
Epoch: 011, Train Acc: 0.8150, Test Acc: 0.7830
Epoch: 012, Train Acc: 0.7633, Test Acc: 0.7308
Epoch: 013, Train Acc: 0.8433, Test Acc: 0.7995
Epoch: 014, Train Acc: 0.7200, Test Acc: 0.6648
Epoch: 015, Train Acc: 0.8683, Test Acc: 0.8297
Epoch: 016, Train Acc: 0.8600, Test Acc: 0.8352
Epoch: 017, Train Acc: 0.8467, Test Acc: 0.7912
Epoch: 018, Train Acc: 0.8467, Test Acc: 0.8187
Epoch: 019, Train Acc: 0.8617, Test Acc: 0.8544
Epoch: 020, Train Acc: 0.8483, Test Acc: 0.8159
Epoch: 021, Train Acc: 0.8400, Test Acc:

In [8]:
torch.save(model.state_dict(), 'gnn_model_weights.pth')
torch.save(model, 'gnn_model.pth')

In [9]:
from torch_geometric.explain import Explainer, GNNExplainer


explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    edge_mask_type='object',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',  # Model returns log of probability.
    ),
)

# Generate explanation for the node at index `10`:
explanation = explainer(data.x, data.edge_index, index=10)
print(explanation.edge_mask)
print(explanation.node_mask)

explanation.visualize_feature_importance(top_k=10)

explanation.visualize_graph()

TypeError: Explainer.__init__() got an unexpected keyword argument 'explanation_type'